In [1]:
import numpy as np
import torch
import scipy
import pandas as pd
import matplotlib.pyplot as plt
from icp import ICP_ROT
from nc_score import pose_err
from tqdm import tqdm
import tools
import rpmg
import argparse
from dataset.CameraPoseDataset import CameraPoseDatasetPred
from rpr import find_poses

In [2]:
from dataset.CameraPoseDataset import CameraPoseDatasetPred
dataset_path = '/home/runyi/Data/7Scenes'
cal_labels_file = '/home/runyi/Project/TBCP6D/dataset/7Scenes_0.5/abs_7scenes_pose.csv_chess_cal.csv_results.csv'
test_labels_file = '/home/runyi/Project/TBCP6D/dataset/7Scenes_0.5/abs_7scenes_pose.csv_chess_test.csv_results.csv'
cal_set = CameraPoseDatasetPred(dataset_path, cal_labels_file)
test_set = CameraPoseDatasetPred(dataset_path, test_labels_file)

In [3]:
cal_set.poses.shape, cal_set.pred_poses.shape

((2417, 7), (2417, 7))

In [4]:
icp_rot = ICP_ROT(torch.tensor(cal_set.poses[:, 3:]), torch.tensor(cal_set.pred_poses[:, 3:]))

In [5]:
dataloader = torch.utils.data.DataLoader(test_set, batch_size=1, shuffle=False, num_workers=1)

In [7]:
for idx, minibatch in enumerate(dataloader):
    img = minibatch.get('img')[0].numpy()  # Assuming this retrieves the current image
    pred_pose = minibatch.get('est_pose')  # Predicted pose from the model
    # Convert quaternion part to rotation matrix
    pred_R = tools.compute_rotation_matrix_from_quaternion(pred_pose[:, 3:])
    pred_t = pred_pose[:, :3]
    
    # Construct the predicted pose matrix
    pred_pose_matrix = np.zeros((4, 4))
    pred_pose_matrix[:3, :3] = pred_R[0]
    pred_pose_matrix[:3, 3] = pred_t[0]
    pred_pose_matrix[3, 3] = 1
    
    for idx, cal_img in enumerate(cal_set.imgs):
        # Assume find_poses computes the relative transformation matrix correctly
        relative_pose = c
        # Adjust the predicted pose based on the relative transformation
        adjusted_pred_pose = np.linalg.inv(pred_pose_matrix) @ relative_pose
        # Convert to tensor and move to GPU
        adjusted_pred_pose_tensor = torch.tensor(adjusted_pred_pose, dtype=torch.float).unsqueeze(0).cuda()
        # Split the adjusted pose into rotation and translation
        R_adj = adjusted_pred_pose_tensor[:, :3, :3]
        t_adj = adjusted_pred_pose_tensor[:, :3, 3]
        # Convert rotation matrix back to quaternion
        adj_quaternion = tools.compute_quaternions_from_rotation_matrices(R_adj)
        # Recombine translation and rotation (quaternion) into the pose vector
        adjusted_pose_vector = torch.cat((t_adj, adj_quaternion), dim=1)
        # Get the ground truth pose for comparison
        cal_gt_pose = torch.tensor(cal_set.poses[idx], dtype=torch.float).unsqueeze(0).cuda()
        # Compute positional and orientation errors
        posit_err, orient_err = pose_err(adjusted_pose_vector, cal_gt_pose)
        print(cal_gt_pose, pred_pose, adjusted_pose_vector)
        print(posit_err.item(), orient_err.item())  # Assuming orient_err is a tensor, calling item() to print the value
    


tensor([[-0.1232, -1.1207, -0.9887,  0.9952, -0.0964,  0.0164,  0.0062]],
       device='cuda:0') tensor([[-0.0316, -0.7684, -0.6945,  0.9902, -0.1354,  0.0263, -0.0213]],
       dtype=torch.float64) tensor([[ 0.7582, -1.0329,  0.6459,  0.1210, -0.0184,  0.1070,  0.9867]],
       device='cuda:0')
1.8592230081558228 165.05030822753906
tensor([[-0.1363, -1.1221, -0.9885,  0.9949, -0.0981,  0.0206,  0.0075]],
       device='cuda:0') tensor([[-0.0316, -0.7684, -0.6945,  0.9902, -0.1354,  0.0263, -0.0213]],
       dtype=torch.float64) tensor([[ 0.1277, -0.2856, -0.1440,  0.1097,  0.0324,  0.0431,  0.9925]],
       device='cuda:0')
1.2176604270935059 166.8714599609375
tensor([[-0.1365, -1.1211, -0.9898,  0.9950, -0.0970,  0.0202,  0.0084]],
       device='cuda:0') tensor([[-0.0316, -0.7684, -0.6945,  0.9902, -0.1354,  0.0263, -0.0213]],
       dtype=torch.float64) tensor([[-0.0479,  0.1508,  0.2993,  0.0687,  0.0422, -0.0625,  0.9948]],
       device='cuda:0')
1.813137173652649 171.819213867

KeyboardInterrupt: 

In [9]:
cal_pose = cal_set.poses[0]
cal_img = cal_set.imgs[0]
cal_pred_pose = cal_set.pred_poses[0]
cal_img_path = cal_set.img_paths[0]

test_pose = test_set.poses[0]
test_img = test_set.imgs[0]
test_pred_pose = test_set.pred_poses[0]
test_img_path = test_set.img_paths[0]

In [10]:
cal_img_path, test_img_path

('/home/runyi/Data/7Scenes/chess/seq-01/frame-000000.color.png',
 '/home/runyi/Data/7Scenes/chess/seq-03/frame-000000.color.png')

In [11]:
test_pose, test_pred_pose

(array([-0.0316476 , -0.76844001, -0.69450545,  0.99021727, -0.13535483,
         0.02634406, -0.02132878]),
 array([-0.0316476 , -0.76844001, -0.69450545,  0.99021727, -0.13535483,
         0.02634406, -0.02132878]))

In [14]:
cal_pose, cal_pred_pose

(array([-0.1232336, -1.1206967, -0.9887058,  0.9951854, -0.0964216,
         0.016435 ,  0.0062336]),
 array([-0.16338812, -1.14563072, -1.01410532,  0.99507397, -0.0974216 ,
         0.01593301,  0.00910558]))

In [16]:
relative_T = find_poses(cal_img, test_img)
relative_T

array([[ 0.9922245 , -0.00755507,  0.12423152, -0.99934721],
       [ 0.03731863,  0.97028924, -0.23905252, -0.0010995 ],
       [-0.11873445,  0.24182992,  0.9630267 ,  0.03611011],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [21]:
# compute the relative pose

def compute_relative_pose(pose1, pose2):
    '''
    pose1 and pose2 are nx7 tensors, where the first 3 elements are the translation and the last 4 elements are the quaternion.
    '''
    # Assuming `tools.compute_rotation_matrix_from_quaternion` is correctly defined elsewhere
    R1 = tools.compute_rotation_matrix_from_quaternion(pose1[:, 3:])
    R2 = tools.compute_rotation_matrix_from_quaternion(pose2[:, 3:])
    t1 = pose1[:, :3].unsqueeze(-1)  # Ensure t1 is nx3x1 for correct broadcasting in matrix operations
    t2 = pose2[:, :3].unsqueeze(-1)  # Ensure t2 is nx3x1

    # Compute the relative rotation
    relative_R = R2.bmm(R1.transpose(1, 2))
    
    # Compute the relative translation
    relative_t = t2 - relative_R.bmm(t1)

    # Flatten relative_t back to nx3 for concatenation
    relative_t = relative_t.squeeze(-1)

    # For returning, you may want to convert relative_R back to quaternions and concatenate with relative_t
    # Assuming `tools.compute_quaternion_from_rotation_matrix` is a function that converts rotation matrices to quaternions
    relative_quaternions = tools.compute_quaternions_from_rotation_matrices(relative_R)
    relative_pose = torch.cat((relative_t, relative_quaternions), dim=1)

    return relative_pose

In [22]:
compute_relative_pose(torch.tensor(cal_pose).unsqueeze(0).cuda(), torch.tensor(test_pose).unsqueeze(0).cuda())

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!